In [1]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
import kagglehub

print("All libraries imported successfully.")

# --- 1. DOWNLOAD DATASET ---
print("Downloading dataset from KaggleHub...")
path_to_dataset = kagglehub.dataset_download("leoscode/wound-segmentation-images")
print(f"Dataset downloaded to: {path_to_dataset}")

# --- 2. PREPARE FILE PATHS ---
base_path = os.path.join(str(path_to_dataset), 'data_wound_seg')
train_images_dir = os.path.join(base_path, 'train_images')
train_masks_dir = os.path.join(base_path, 'train_masks')

train_image_files = sorted(os.listdir(train_images_dir))
train_mask_files = sorted(os.listdir(train_masks_dir))

full_train_image_paths = [os.path.join(train_images_dir, f) for f in train_image_files]
full_train_mask_paths = [os.path.join(train_masks_dir, f) for f in train_mask_files]
print(f"Found {len(train_image_files)} training images and masks.")

# --- 3. BUILD THE DATA PIPELINE ---
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 16

def parse_image_and_mask(image_path, mask_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, [IMG_HEIGHT, IMG_WIDTH])

    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=1)
    mask = tf.image.resize(mask, [IMG_HEIGHT, IMG_WIDTH])
    mask = tf.cast(mask > 0.5, dtype=tf.float32)
    return image, mask

dataset = tf.data.Dataset.from_tensor_slices((full_train_image_paths, full_train_mask_paths))
dataset = dataset.map(parse_image_and_mask, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
print("TensorFlow data pipeline created successfully.")

# --- 4. BUILD THE U-NET MODEL ---
def build_unet_model(input_shape=(224, 224, 3)):
    inputs = layers.Input(input_shape)
    # Encoder
    c1 = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(inputs)
    p1 = layers.MaxPooling2D((2, 2))(c1)
    c2 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(p1)
    p2 = layers.MaxPooling2D((2, 2))(c2)
    # Bottleneck
    b = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(p2)
    # Decoder
    u2 = layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(b)
    u2 = layers.concatenate([u2, c2])
    c7 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(u2)
    u3 = layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c7)
    u3 = layers.concatenate([u3, c1])
    c8 = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(u3)
    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c8)
    model = models.Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = build_unet_model()
model.summary()

# --- 5. TRAIN THE MVP MODEL ---
print("\nStarting model training for the MVP...")
EPOCHS = 50+
history = model.fit(dataset, epochs=EPOCHS)
print("MVP model training complete!")

model.save('wound_segmentation_model_v1.keras')
print("Model successfully saved as wound_segmentation_model_v1.keras")

d:\hackathon_project\backend\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


All libraries imported successfully.
Dataset downloaded to: C:\Users\Chandan\.cache\kagglehub\datasets\leoscode\wound-segmentation-images\versions\1
Found 2208 training images and masks.
TensorFlow data pipeline created successfully.


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 224, 224,  │        448 │ input_layer[0][0] │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 112, 112,  │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 112, 112,  │      4,640 │ max_pooling2d[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 56, 56,    │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 56, 56,    │     18,496 │ max_pooling2d_1[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose    │ (None, 112, 112,  │      8,224 │ conv2d_2[0][0]    │
│ (Conv2DTranspose)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 112, 112,  │          0 │ conv2d_transpose… │
│ (Concatenate)       │ 64)               │            │ conv2d_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 112, 112,  │     18,464 │ concatenate[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_1  │ (None, 224, 224,  │      2,064 │ conv2d_3[0][0]    │
│ (Conv2DTranspose)   │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 224, 224,  │          0 │ conv2d_transpose… │
│ (Concatenate)       │ 32)               │            │ conv2d[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 224, 224,  │      4,624 │ concatenate_1[0]… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 224, 224,  │         17 │ conv2d_4[0][0]    │
│                     │ 1)                │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 56,977 (222.57 KB)

 Trainable params: 56,977 (222.57 KB)

 Non-trainable params: 0 (0.00 B)


Starting model training for the MVP...
Epoch 1/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 33s 224ms/step - accuracy: 0.9531 - loss: 0.2037
Epoch 2/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 32s 225ms/step - accuracy: 0.9543 - loss: 0.1327
Epoch 3/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 32s 226ms/step - accuracy: 0.9606 - loss: 0.1173
Epoch 4/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 32s 226ms/step - accuracy: 0.9623 - loss: 0.1115
Epoch 5/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 32s 226ms/step - accuracy: 0.9618 - loss: 0.1120
Epoch 6/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 32s 229ms/step - accuracy: 0.9624 - loss: 0.1105
Epoch 7/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 32s 226ms/step - accuracy: 0.9626 - loss: 0.1087
Epoch 8/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 36s 258ms/step - accuracy: 0.9635 - loss: 0.1071
Epoch 9/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 37s 262ms/step - accuracy: 0.9634 - loss: 0.1042
Epoch 10/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 40s 284ms/step - accuracy: 0.9627 - loss: 0.1093
Epoch 11/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 37s 262ms/step - accuracy: 0